In [3]:
import pylab
from matplotlib import gridspec
from sklearn.datasets import make_classification
import numpy as np
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pickle
import os


# pick the seed for reproducability - change it to explore the effects of random variations
np.random.seed(1)
import random

In [4]:
def train_graph(positive_examples, negative_examples, num_iterations = 100):
    num_dims = positive_examples.shape[1]
    weights = np.zeros((num_dims,1)) # инициализируем веса
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]
    
    report_frequency = 15;
    snapshots = []
    
    for i in range(num_iterations):
        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)

        z = np.dot(pos, weights)   
        if z < 0:
            weights = weights + pos.reshape(weights.shape)

        z  = np.dot(neg, weights)
        if z >= 0:
            weights = weights - neg.reshape(weights.shape)
            
        if i % report_frequency == 0:             
            pos_out = np.dot(positive_examples, weights)
            neg_out = np.dot(negative_examples, weights)        
            pos_correct = (pos_out >= 0).sum() / float(pos_count)
            neg_correct = (neg_out < 0).sum() / float(neg_count)
            snapshots.append((np.copy(weights),(pos_correct+neg_correct)/2.0))

    return np.array(snapshots)

In [8]:
with open('mnist.pkl', 'rb') as mnist_pickle:
    MNIST = pickle.load(mnist_pickle)

# Классификация десяти цифр

Обучение на тестовой выборке (80% от всей)

In [9]:
def set_mnist_by_label(label):
    indices = [i for i, j in enumerate(MNIST['Train']['Labels']) 
                          if j == label and i < MNIST['Train']['Labels'].shape[0]*0.8]
    negative_indices = [i for i, j in enumerate(MNIST['Train']['Labels']) 
                          if j != label and i < MNIST['Train']['Labels'].shape[0]*0.8]

    images = MNIST['Train']['Features'][indices]
    negative_images = MNIST['Train']['Features'][negative_indices]
    
    return images, negative_images

In [10]:
def tenPerseptrons():
    snapshots = []
    for i in range(10):
        pos, neg = set_mnist_by_label(i)
        snapshot = train_graph(pos, neg, 1000)
        snapshots.append(snapshot[-1])
    return snapshots

In [11]:
snapshots = tenPerseptrons()
#показать верятности

for i in range(len(snapshots)):
    print(i, snapshots[i][1])
    
def predictTenPerseptrons(X, snapshots):
    predictions = []
    for snapshot in snapshots:
        predictions.append(np.dot(X, snapshot[0]))
    return predictions.index(max(predictions)), max(predictions)

C:\Users\prota\AppData\Local\Temp/ipykernel_13800/2513895739.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(snapshots)


0 0.912785974000901
1 0.9725626007199234
2 0.9184942006478013
3 0.8622719084863971
4 0.8939015800388452
5 0.8036653141256007
6 0.9562796092545924
7 0.9450055821770554
8 0.80818274503932
9 0.8635564010223323


# Тестировние на отставшися 20%

In [12]:
faults = 0
for ind in range(int(MNIST['Train']['Labels'].shape[0]*0.8), MNIST['Train']['Features'].shape[0]):
    prediction, result = predictTenPerseptrons(MNIST['Train']['Features'][ind], snapshots)
    if (prediction != MNIST['Train']['Labels'][ind]):
        faults += 1

print('вероятность правильного ответа:', ((MNIST['Train']['Labels'].shape[0]*0.2) - faults) / (MNIST['Train']['Labels'].shape[0]*0.2))

вероятность правильного ответа: 0.7698809523809523
